This is currently under construction.

This notebook contains the functions that are called inside the rubix pipeline.

# Config

contains all the information needed to run the pipeline. Those are run specfic configurations.

In [ ]:
import os
config = {
    "pipeline":{"name": "calc_ifu"},
    
    "logger": {
        "log_level": "DEBUG",
        "log_file_path": None,
        "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    },
    "data": {
        "name": "IllustrisAPI",
        "args": {
            "api_key": os.environ.get("ILLUSTRIS_API_KEY"),
            "particle_type": ["stars"],
            "cube_type": ["stars"],
            "simulation": "TNG50-1",
            "snapshot": 99,
            "save_data_path": "data",
        },
        
        "load_galaxy_args": {
        "id": 12,
        "reuse": True,
        }
    },
    "simulation": {
        "name": "IllustrisTNG",
        "args": {
            "path": "data/galaxy-id-12.hdf5",
        },
    
    },
    "output_path": "output",

    "telescope":
        {"name": "MUSE",
         "psf": {"name": "gaussian", "size": 5, "sigma": 0.6},
         "lsf": {"sigma": 0.5},
         "noise": {"signal_to_noise": 1,"noise_distribution": "normal"},},
        
    "cosmology":
        {"name": "PLANCK15"},
        
    "galaxy":
        {"dist_z": 0.1,
         "rotation": {"type": "edge-on"},
        },
    "ssp": {
        "template": {
            "name": "BruzualCharlot2003"
        },
    },    
}

In [ ]:
# NBVAL_SKIP
from rubix.core.data import convert_to_rubix, prepare_input

convert_to_rubix(config) # Convert the config to rubix format and store in output_path folder
rubixdata = prepare_input(config) # Prepare the input for the pipeline

In [ ]:
# NBVAL_SKIP
import matplotlib.pyplot as plt
# Make a scatter plot 
n_subset = 10000
subset = rubixdata.stars.coords[:n_subset]

plt.scatter(subset[:,0], subset[:,1], s=1)


Get the function that does the binning

In [ ]:
from rubix.core.rotation import get_galaxy_rotation

In [ ]:
print(rubixdata.stars.coords)

In [ ]:
rotate = get_galaxy_rotation(config)
rubixdata = rotate(rubixdata)

In [ ]:
print(rubixdata.stars.coords)

In [ ]:
from rubix.core.telescope import get_filter_particles
filter_particles = get_filter_particles(config)

rubixdata = filter_particles(rubixdata)

In [ ]:
# NBVAL_SKIP
from rubix.core.telescope import get_spaxel_assignment
bin_particles = get_spaxel_assignment(config)

rubixdata = bin_particles(rubixdata)

print(rubixdata.stars.pixel_assignment)
print(rubixdata.stars.spatial_bin_edges)

In [ ]:
from rubix.core.data import get_reshape_data
reshape_data = get_reshape_data(config)

rubixdata = reshape_data(rubixdata)

In [ ]:
from rubix.core.ifu import get_calculate_spectra
calcultae_spectra = get_calculate_spectra(config)

rubixdata = calcultae_spectra(rubixdata)
print(rubixdata.stars.spectra)

In [ ]:
from rubix.core.ifu import get_scale_spectrum_by_mass

scale_spectrum_by_mass = get_scale_spectrum_by_mass(config)

rubixdata = scale_spectrum_by_mass(rubixdata)
print(rubixdata.stars.spectra)

In [ ]:
from rubix.core.ifu import get_doppler_shift_and_resampling

doppler_shift_and_resampling = get_doppler_shift_and_resampling(config)

rubixdata = doppler_shift_and_resampling(rubixdata)
print(rubixdata.stars.spectra)

In [ ]:
from rubix.core.pipeline import RubixPipeline 

pipe = RubixPipeline(config)

wave = pipe.telescope.wave_seq
print(wave)
print(rubixdata.stars.spectra[0][0][:])

plt.plot(wave, rubixdata.stars.spectra[0][0][:])
plt.plot(wave, rubixdata.stars.spectra[0][1][:])

In [ ]:
from rubix.core.ifu import get_calculate_datacube
calculate_datacube = get_calculate_datacube(config)

rubixdata = calculate_datacube(rubixdata)
print(rubixdata.stars.datacube)

In [ ]:
datacube = rubixdata.stars.datacube
img = datacube.sum(axis=2)
plt.imshow(img, origin="lower")

In [ ]:
from rubix.core.psf import get_convolve_psf
convolve_psf = get_convolve_psf(config)

rubixdata = convolve_psf(rubixdata)

In [ ]:
datacube = rubixdata.stars.datacube
img = datacube.sum(axis=2)
plt.imshow(img, origin="lower")

In [ ]:
plt.plot(wave, datacube[12,12,:])
plt.plot(wave, datacube[0,0,:])

In [ ]:
from rubix.core.lsf import get_convolve_lsf
convolve_lsf = get_convolve_lsf(config)

rubixdata = convolve_lsf(rubixdata)

plt.plot(wave, rubixdata.stars.datacube[12,12,:])
plt.plot(wave, rubixdata.stars.datacube[0,0,:])
print(rubixdata.stars.datacube)

In [ ]:
from rubix.core.noise import get_apply_noise

apply_noise = get_apply_noise(config)

rubixdata = apply_noise(rubixdata)

datacube = rubixdata.stars.datacube
img = datacube.sum(axis=2)
plt.imshow(img, origin="lower")

In [ ]:
plt.plot(wave, rubixdata.stars.datacube[12,12,:])
plt.plot(wave, rubixdata.stars.datacube[0,0,:])